In [1]:
import asyncio
import websockets
import json
from datetime import datetime, timezone
import pandas as pd

In [9]:
async def connect_ais_stream(api_key, nb_values, output_file):
    data = []
    async with websockets.connect("wss://stream.aisstream.io/v0/stream") as websocket:
        subscribe_message = {
            "APIKey": api_key,
            "BoundingBoxes": [[[-90, -180], [90, 180]]],  # Limite géographique, [[[-90, -180], [90, 180]]] pour tout le monde
            "FilterMessageTypes": ["PositionReport"]  # Filtre
        }

        subscribe_message_json = json.dumps(subscribe_message)
        await websocket.send(subscribe_message_json)

        async for message_json in websocket:
            message = json.loads(message_json)
            message_type = message["MessageType"]
            print(message)
            
            if message_type == "PositionReport":
                ais_message = message['Message']['PositionReport']
                metadata = message['MetaData']
                
                ship_data = {
                    "timestamp": datetime.now(timezone.utc),
                    "MMSI": metadata['MMSI'],  # ID du navire
                    "ShipName": metadata['ShipName'],  # Nom du navire
                    "UserID": ais_message['UserID'],
                    "Latitude": ais_message['Latitude'],
                    "Longitude": ais_message['Longitude'],
                    "SOG": ais_message['Sog'],  # Vitesse au sol
                    "Cog": ais_message['Cog'],  # Cap
                    "NavigationalStatus": ais_message['NavigationalStatus'],  # Statut de navigation
                    "PositionAccuracy": ais_message['PositionAccuracy'],  # Position
                    "time_utc": metadata['time_utc'],  # Temps UTC
                    "RateOfTurn": ais_message['RateOfTurn'],  # Rotation
                    "TrueHeading": ais_message['TrueHeading'],  # Cap réel
                }
                data.append(ship_data)
                if len(data) >= nb_values:  
                    break

        with open(output_file, 'w') as f:
            json.dump(data, f, indent=4) 

        df = pd.DataFrame(data)
        return df

async def run_ais_stream(api_key, num_values, output_file):
    return await connect_ais_stream(api_key, num_values, output_file)

In [10]:
output_file_path = 'ais_data.json'
df = await run_ais_stream("e66dea5edc96977c45bd4ddba1e1b34acf718a04", 10, output_file_path)
df

,timestamp,MMSI,ShipName,UserID,Latitude,Longitude,SOG,Cog,NavigationalStatus,PositionAccuracy,time_utc,RateOfTurn,TrueHeading
0,2024-10-16 11:02:30.296596+00:00,273433280,KAPITAN SOKOLOV,273433280,42.786100,132.924067,9.2,61.4,0,True,2024-10-16 11:02:31.219806292 +0000 UTC,0,59
1,2024-10-16 11:02:30.335351+00:00,244066650,HET LAND GOESTING,244066650,52.374107,4.907445,0.0,0.0,0,False,2024-10-16 11:02:31.221278383 +0000 UTC,0,0
2,2024-10-16 11:02:30.335351+00:00,244066650,HET LAND GOESTING,244066650,52.374108,4.907445,0.0,360.0,0,True,2024-10-16 11:02:31.221574771 +0000 UTC,-128,511
3,2024-10-16 11:02:30.335351+00:00,660006100,SAINTE-MARIE,660006100,-20.931400,55.287218,0.0,360.0,7,True,2024-10-16 11:02:31.22197828 +0000 UTC,-128,511
4,2024-10-16 11:02:30.335351+00:00,258263500,LOS117,258263500,59.082093,10.452992,0.0,187.1,0,False,2024-10-16 11:02:31.222420542 +0000 UTC,0,253
...,...,...,...,...,...,...,...,...,...,...,...,...,...
249995,2024-10-16 11:17:11.490662+00:00,211211730,BERLIN,211211730,53.530133,8.154983,0.0,224.0,5,True,2024-10-16 11:17:12.412245881 +0000 UTC,0,2
249996,2024-10-16 11:17:11.490662+00:00,258542000,VESTFJORD,258542000,58.336670,8.596278,0.0,243.6,0,False,2024-10-16 11:17:12.412858483 +0000 UTC,0,300
249997,2024-10-16 11:17:11.490662+00:00,211242870,LOESCHBOOT REPSOLD,211242870,53.527150,10.057712,0.0,213.1,5,False,2024-10-16 11:17:12.413970504 +0000 UTC,-128,511
249998,2024-10-16 11:17:11.491659+00:00,308015000,MOL CHARISMA,308015000,48.987425,-123.290598,20.8,134.6,0,False,2024-10-16 11:17:12.414671704 +0000 UTC,0,135


In [12]:
#df.to_csv("data.csv")